<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/adv_cv/class_2/10%20-%20%20Laborat%C3%B3rio/C3_W2_Lab_1_Simple_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Visão computacional avançada com TensorFlow](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow?specialization=tensorflow-advanced-techniques) de [Laurence Moroney](https://laurencemoroney.com/) e [Andrew Ng](https://www.coursera.org/instructor/andrewng) , [DeepLearning.AI](https://www.deeplearning.ai/)

# Detecção simples de objetos no Tensorflow

Neste laboratório, você aprenderá a usar os modelos de detecção de objetos disponíveis no [Tensorflow Hub](https://www.tensorflow.org/hub).

Nas seções a seguir, você vai:
* explorar o Tensorflow Hub para modelos de detecção de objetos
* carregar os modelos em seu espaço de trabalho
* pré-processar uma imagem para inferência 
* executar a inferência nos modelos e inspecionar a saída

## Imports

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

2024-04-26 19:42:32.606990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 19:42:32.607350: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-26 19:42:32.609574: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-26 19:42:32.636245: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Baixar o modelo do Tensorflow Hub

O Tensorflow Hub é um repositório de modelos de aprendizado de máquina treinados que você pode reutilizar em seus próprios projetos. 
- Você pode ver os domínios cobertos [aqui](https://tfhub.dev/) e suas subcategorias. 
- Para este laboratório, você deverá examinar a subcategoria[`image-object-detection`](https://tfhub.dev/s?module-type=image-object-detection). 
- Você pode selecionar um modelo para ver mais informações sobre ele e copiar o URL para fazer o download em seu espaço de trabalho. 
- Utilizaremos um [inception resnet version 2](https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1)
- Você também pode modificar a célula a seguir para escolher o outro modelo: [ssd mobilenet version 2](https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2)

In [2]:
# você pode trocar as linhas comentadas aqui para escolher o outro modelo

# Início da versão 2 do resnet
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# Em vez disso, você pode escolher a versão 2 do ssd mobilenet e comparar os resultados
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

#### Carregar o modelo

Em seguida, você carregará o modelo especificado pelo `module_handle`.
- Isso levará alguns minutos para carregar o modelo.

In [3]:
model = hub.load(module_handle)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2024-04-26 19:43:25.643865: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-26 19:43:25.646051: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


#### Escolha a assinatura padrão

Alguns modelos no hub do Tensorflow podem ser usados para diferentes tarefas.

Portanto, a documentação de cada modelo deve mostrar qual *assinatura* usar ao executar o modelo. 
- Se quiser ver se um modelo tem mais de uma assinatura, você pode fazer algo como `print(hub.load(module_handle).signatures.keys())`.

Aqui os modelos que você usará têm apenas a assinatura `default`, portanto, você não precisa se preocupar com outros tipos.

In [4]:
# Dê uma olhada nas assinaturas disponíveis para esse modelo específico
model.signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction () -> Dict[['detection_class_labels', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None)], ['detection_scores', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)], ['detection_boxes', TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)], ['detection_class_entities', TensorSpec(shape=(None, 1), dtype=tf.string, name=None)], ['detection_class_names', TensorSpec(shape=(None, 1), dtype=tf.string, name=None)]] at 0x7FCC9848FE90>}))

Escolha a assinatura "padrão" para seu detector de objetos.

- Para modelos de detecção de objetos, sua assinatura "padrão" aceitará um lote de tensores de imagem e produzirá um dicionário que descreve os objetos detectados, que é o que você deseja aqui.

In [5]:
detector = model.signatures['default']

### download_and_resize_image

Essa função faz o download de uma imagem especificada por uma determinada "url", pré-processa-a e, em seguida, salva-a no disco.

In [7]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Obtém uma imagem on-line, redimensiona-a e salva-a localmente.
    
    Args:
        url (string) -- link para a imagem
        new_width (int) -- tamanho em pixels usado para redimensionar a largura da imagem
        new_height (int) -- tamanho em pixels usado para redimensionar o comprimento da imagem
        
    Retorna:
        (string) -- caminho para a imagem salva
    '''
    
    
    # cria um arquivo temporário que termina com ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # abre o URL fornecido
    response = urlopen(url)
    
    # lê a imagem obtida do URL
    image_data = response.read()
    
    # Coloca os dados da imagem no buffer de memória
    image_data = BytesIO(image_data)
    
    # abre a imagem
    pil_image = Image.open(image_data)
    
    # Redimensiona a imagem. Será cortada se a proporção for diferente.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    
    # converte para o espaço de cores RGB
    pil_image_rgb = pil_image.convert("RGB")
    
   # salva a imagem no arquivo temporário criado anteriormente
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Imagem baixada para %s." % filename)
    
    return filename

### Baixar e pré-processar uma imagem

Agora, usando `download_and_resize_image`, você pode obter uma imagem de amostra on-line e salvá-la localmente. 
- Forneci uma URL para você, mas fique à vontade para escolher outra imagem para passar pelo detector de objetos.
- Você pode usar a largura e a altura originais da imagem, mas fique à vontade para modificá-la e ver os resultados obtidos.

In [8]:
# Você pode escolher um URL diferente que aponte para uma imagem de sua escolha
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# Faça o download da imagem e use a altura e a largura originais
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'

### run_detector

This function will take in the object detection model `detector` and the path to a sample image, then use this model to detect objects and display its predicted class categories and detection boxes.
- run_detector uses `load_image` to convert the image into a tensor.

In [ ]:
def load_img(path):
    '''
    Loads a JPEG image and converts it to a tensor.
    
    Args:
        path (string) -- path to a locally saved JPEG image
    
    Returns:
        (tensor) -- an image tensor
    '''
    
    # read the file
    img = tf.io.read_file(path)
    
    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    
    return img


def run_detector(detector, path):
    '''
    Runs inference on a local file using an object detection model.
    
    Args:
        detector (model) -- an object detection model loaded from TF Hub
        path (string) -- path to an image saved locally
    '''
    
    # load an image tensor from a local file path
    img = load_img(path)

    # add a batch dimension in front of the tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    
    # run inference using the model
    result = detector(converted_img)

    # save the results in a dictionary
    result = {key:value.numpy() for key,value in result.items()}

    # print results
    print("Found %d objects." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])


### Run inference on the image

You can run your detector by calling the `run_detector` function. This will print the number of objects found followed by three lists: 

* The detection scores of each object found (i.e. how confident the model is), 
* The classes of each object found, 
* The bounding boxes of each object

You will see how to overlay this information on the original image in the next sections and in this week's assignment!

In [ ]:
# runs the object detection model and prints information about the objects found
run_detector(detector, downloaded_image_path)